In [5]:
#Slope calculus

#import network
from owslib.wcs import WebCoverageService
import osmnx as ox
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import LineString
from rasterio.windows import Window
from geopy.distance import geodesic
import matplotlib.pyplot as plt
from io import BytesIO

# Get the network
G = ox.graph_from_place('Montereale Valcellina, Italy', network_type='all')
edges = ox.graph_to_gdfs(G, nodes=False)

# Filter the major roads
major_roads = ['primary', 'primary_link', 'secondary', 'secondary_link', 'tertiary',
               'tertiary_link', 'trunk', 'trunk_link', 'residential', 'cycleway',
               'living_street', 'unclassified', 'motorway', 'motorway_link',
               'pedestrian', 'steps', 'track']
edges = edges[edges['highway'].isin(major_roads)]

# Define the WCS service
#wcs_url = 'http://tinitaly.pi.ingv.it/TINItaly_1_1/wcs?service=WCS&request=getCapabilities'
#wcs = WebCoverageService(wcs_url, version='1.0.0')

# List the coverages available on the service
#list(wcs.contents)

# Assuming TINItaly_DEM is the coverage you want, get that coverage
# Make sure to select the right CRS and bounding box
#output = wcs.getCoverage(identifier='TINItaly_DEM', crs='EPSG:4326', bbox=list(edges.total_bounds), format='GeoTIFF')

# Load the DEM with WCS
#dem = rasterio.open(BytesIO(output.read()))


In [7]:
import rasterio
from pyproj import CRS, Transformer
print(edges.crs)

#SRTM DEM scenario
# Set the path to the DEM file
dem_path = '/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/MonterealeValc_SRTM.tif'

# Load the DEM
dem = rasterio.open(dem_path)
# Define a Transformer from EPSG:4326 to a local projection
transformer = Transformer.from_crs(CRS('EPSG:4326'), CRS('EPSG:32632'), always_xy=True)

# Calculate the slope
def calc_slope(row):
    coords = row['geometry'].coords.xy
    start = (coords[0][0], coords[1][0])
    end = (coords[0][-1], coords[1][-1])

    start_elevation = float(list(dem.sample([start]))[0][0])
    end_elevation = float(list(dem.sample([end]))[0][0])
    dz = end_elevation - start_elevation

    if abs(dz) > 100:  # Adjust this value based on what you consider to be an "outlier"
        return 0

    start_x, start_y = transformer.transform(*start)
    end_x, end_y = transformer.transform(*end)
    dx = end_x - start_x
    dy = end_y - start_y
    distance = np.sqrt(dx ** 2 + dy ** 2)

    slope = (dz / distance) * 100 if distance != 0 else 0
    return slope

# Apply function to each row in the GeoDataFrame
edges['slope'] = edges.apply(calc_slope, axis=1)

# Replace NaN slope values with 0
edges['slope'] = edges['slope'].fillna(0)

# Classify slopes
edges['slope_class'] = pd.cut(edges['slope'],
                              bins=[-30, 3, 5, 8, 10, 20, np.inf],
                              labels=["0-3: flat", "3-5: mild", "5-8: medium",
                                      "8-10: hard", "10-20: extreme", ">20: impossible"],
                              right=False)
edges['slope_class'] = edges['slope_class'].fillna("0-3: flat")

# Calculate the proportion of each slope class
slope_class_distribution = round(edges['slope_class'].value_counts(normalize=True) * 100, 1)
print(slope_class_distribution)
print(dem.crs)


epsg:4326
0-3: flat          86.1
3-5: mild           5.4
5-8: medium         3.3
10-20: extreme      2.7
8-10: hard          1.8
>20: impossible     0.7
Name: slope_class, dtype: float64
EPSG:4326


In [9]:
#Tinitaly scenario
import rasterio
from pyproj import CRS, Transformer
print(edges.crs)

# Set the path to the DEM file
dem_path = '/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/w51075_s10.tif'

# Load the DEM
dem = rasterio.open(dem_path)

# Define a Transformer from EPSG:4326 to DEM's CRS
transformer = Transformer.from_crs(CRS('EPSG:4326'), CRS.from_epsg(dem.crs.to_epsg()), always_xy=True)

# Transform road network bounding box to match the CRS of the DEM
road_bounds_utm = transformer.transform_bounds(*edges.total_bounds)

# Check if the road network bounding box is within the DEM bounding box
assert (dem.bounds.left <= road_bounds_utm[0] and
        dem.bounds.bottom <= road_bounds_utm[1] and
        dem.bounds.right >= road_bounds_utm[2] and
        dem.bounds.top >= road_bounds_utm[3]), "Road network bounding box is not within the DEM bounding box!"

# Compute the average length of the road segments
average_length = edges.length.mean()

def calc_slope(row):
    coords = row['geometry'].coords.xy
    elevation = []

    # If the road segment is shorter than the average, use the start and end points only
    if row['geometry'].length <= average_length:
        points = [(coords[0][0], coords[1][0]), (coords[0][-1], coords[1][-1])]
    else:
        points = [(coords[0][i], coords[1][i]) for i in range(len(coords[0]))]

    for i in range(len(points)-1):
        start = points[i]
        end = points[i+1]

        start_x, start_y = transformer.transform(*start)
        end_x, end_y = transformer.transform(*end)

        # Check if the start and end points are within the bounds of the DEM
        if (start_x < dem.bounds.left or start_x > dem.bounds.right or
            start_y < dem.bounds.bottom or start_y > dem.bounds.top or
            end_x < dem.bounds.left or end_x > dem.bounds.right or
            end_y < dem.bounds.bottom or end_y > dem.bounds.top):
            return np.nan

        start_elevation = float(list(dem.sample([(start_x, start_y)]))[0][0])
        if start_elevation == dem.nodata:
            start_elevation = np.nan
        end_elevation = float(list(dem.sample([(end_x, end_y)]))[0][0])
        if end_elevation == dem.nodata:
            end_elevation = np.nan

        dz = end_elevation - start_elevation

        dx = end_x - start_x
        dy = end_y - start_y
        distance = np.sqrt(dx ** 2 + dy ** 2)

        slope = (dz / distance) * 100 if distance != 0 else 0
        elevation.append(slope)

    return np.nanmean(elevation)  # calculate mean ignoring NaNs. 


# Apply function to each row in the GeoDataFrame
edges['slope'] = edges.apply(calc_slope, axis=1)

# Replace NaN slope values with 0
edges['slope'] = edges['slope'].fillna(0)

# Classify slopes
edges['slope_class'] = pd.cut(edges['slope'],
                          bins=[-30, 3, 5, 8, 10, 20, np.inf],
                          labels=["0-3: flat", "3-5: mild", "5-8: medium",
                                  "8-10: hard", "10-20: extreme", ">20: impossible"],
                          right=False)
#edges['slope_class'] = edges['slope_class'].fillna("0-3: flat")

# Calculate the proportion of each slope class
slope_class_distribution = round(edges['slope_class'].value_counts(normalize=True) * 100, 1)
print(slope_class_distribution)
# # Plot roads and DEM to check alignment
# fig, ax = plt.subplots(1, 1, figsize=(12, 12))

# # Plot DEM
# dem_array = dem.read(1)
# ax.imshow(dem_array, cmap='terrain', extent=[dem.bounds.left, dem.bounds.right, dem.bounds.bottom, dem.bounds.top])

# # Plot roads
# edges_projected = edges.to_crs(dem.crs)
# edges_projected.plot(ax=ax, color='black')

# plt.show()
print(dem.crs)


epsg:4326


/var/folders/9_/45rjymz11pnd9_lk8d45f5t40000gn/T/ipykernel_25272/4063944832.py:25: UserWarning: Geometry is in a geographic CRS. Results from 'length' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  average_length = edges.length.mean()


0-3: flat          91.0
3-5: mild           3.5
5-8: medium         2.3
10-20: extreme      1.8
8-10: hard          1.1
>20: impossible     0.3
Name: slope_class, dtype: float64
EPSG:32632


In [28]:
#slope calc using Copernicus dem

import osmnx as ox
import folium
import geopandas as gpd
import pandas as pd
import numpy as np
import rasterio
from shapely.geometry import LineString
from pyproj import CRS, Transformer
from rasterio.windows import Window
from geopy.distance import geodesic

# Get the network
G = ox.graph_from_place('Montereale Valcellina, Italy', network_type='all')
edges = ox.graph_to_gdfs(G, nodes=False)

# Filter the major roads
major_roads = ['primary', 'primary_link', 'secondary', 'secondary_link', 'tertiary',
               'tertiary_link', 'trunk', 'trunk_link', 'residential', 'cycleway',
               'living_street', 'unclassified', 'motorway', 'motorway_link',
               'pedestrian', 'steps', 'track', 'cycleway']
edges = edges[edges['highway'].isin(major_roads)]

# Set the path to the DEM file
dem_path = '/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/output_COP30.tif'

# Load the DEM
dem = rasterio.open(dem_path)

# Define a Transformer from EPSG:4326 to a local projection
transformer = Transformer.from_crs(CRS('EPSG:4326'), CRS('EPSG:32632'), always_xy=True)

# Calculate the slope
def calc_slope(row):
    coords = row['geometry'].coords.xy
    start = (coords[0][0], coords[1][0])
    end = (coords[0][-1], coords[1][-1])

    start_elevation = float(list(dem.sample([start]))[0][0])
    end_elevation = float(list(dem.sample([end]))[0][0])
    dz = end_elevation - start_elevation

    if abs(dz) > 100:  # Adjust this value based on what you consider to be an "outlier"
        return 0

    start_x, start_y = transformer.transform(*start)
    end_x, end_y = transformer.transform(*end)
    dx = end_x - start_x
    dy = end_y - start_y
    distance = np.sqrt(dx ** 2 + dy ** 2)

    slope = (dz / distance) * 100 if distance != 0 else 0
    return slope

# Apply function to each row in the GeoDataFrame
edges['slope'] = edges.apply(calc_slope, axis=1)

# Replace NaN slope values with 0
edges['slope'] = edges['slope'].fillna(0)

# Classify slopes
edges['slope_class'] = pd.cut(edges['slope'],
                              bins=[-30, 3, 5, 8, 10, 20, np.inf],
                              labels=["0-3: flat", "3-5: mild", "5-8: medium",
                                      "8-10: hard", "10-20: extreme", ">20: impossible"],
                              right=False)
edges['slope_class'] = edges['slope_class'].fillna("0-3: flat")

# Calculate the proportion of each slope class
slope_class_distribution = round(edges['slope_class'].value_counts(normalize=True) * 100, 1)
print(slope_class_distribution)
#print(dem.overviews)

0-3: flat          86.6
3-5: mild           6.0
5-8: medium         2.9
10-20: extreme      2.3
8-10: hard          1.2
>20: impossible     1.0
Name: slope_class, dtype: float64


In [50]:
import folium
# Create a colormap for slope classes
color_palette = ["#267300", "#70A800", "#FFAA00", "#E60000", "#A80000", "#730000"]
slope_classes = ["0-3: flat", "3-5: mild", "5-8: medium", "8-10: hard", "10-20: extreme", ">20: impossible"]
colors = dict(zip(slope_classes, color_palette))

# Calculate the mean of latitudes and longitudes
mean_latitude = edges.geometry.apply(lambda geom: geom.centroid.y).mean()
mean_longitude = edges.geometry.apply(lambda geom: geom.centroid.x).mean()

# Create a folium map centered on the mean of latitudes and longitudes
map_osm = folium.Map(location=[mean_latitude, mean_longitude], zoom_start=11)

# Add slope information to the map
for _, row in edges.iterrows():
    color = colors.get(str(row['slope_class']), "#000000")  # default color is black
    folium.GeoJson(
        row['geometry'], 
        style_function=lambda feature, color=color: {'color': color}  # use default argument to capture color
    ).add_to(map_osm)

# Create a custom legend HTML string
legend_html = '''
<div style="position: fixed; top: 10px; right: 10px; z-index: 1000; background-color: white; padding: 5px; border: 1px solid grey; font-size: 12px;">
<p><b>Slope</b></p>
'''
for slope_class, color in colors.items():
    legend_html += f'<p><i class="fa fa-square" style="color:{color};"></i> {slope_class}</p>'
legend_html += '</div>'

# Add the legend HTML to the map
map_osm.get_root().html.add_child(folium.Element(legend_html))

# Save the map
map_osm.save('/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/monterale.html')

# Display the map 
map_osm


In [12]:
def inspect_dem(dem_path):
    with rasterio.open(dem_path) as dem:
        # Print the DEM's metadata
        print(f"Metadata for {dem_path}:\n{dem.meta}\n")
        
        # Read the DEM's data
        data = dem.read(1)
        
        # Print the min, max, and mean values
        print(f"Min value: {data.min()}")
        print(f"Max value: {data.max()}")
        print(f"Mean value: {data.mean()}\n")
        
        # Return the data for further use
        return data

# Inspect the first DEM
data1 = inspect_dem('/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/MonterealeValc_SRTM.tif')

# Inspect the second DEM
data2 = inspect_dem('/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/w51075_s10.tif')

# Inspect the second DEM
data3 = inspect_dem('/Users/leonardo/Desktop/Tesi/LTSBikePlan/data/MonterealeValce_Cop.tif')


Metadata for /Users/leonardo/Desktop/Tesi/LTSBikePlan/data/MonterealeValc_SRTM.tif:
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32767.0, 'width': 3601, 'height': 3601, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0002777777777777778, 0.0, 11.99986111111111,
       0.0, -0.0002777777777777778, 47.000138888888884)}

Min value: -32767
Max value: 3259
Mean value: -2080.6597223255812

Metadata for /Users/leonardo/Desktop/Tesi/LTSBikePlan/data/w51075_s10.tif:
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -9999.0, 'width': 5010, 'height': 5010, 'count': 1, 'crs': CRS.from_epsg(32632), 'transform': Affine(10.0, 0.0, 749950.0,
       0.0, -10.0, 5150050.0)}

Min value: 20.368610382080078
Max value: 3260.94189453125
Mean value: 891.0731811523438

Metadata for /Users/leonardo/Desktop/Tesi/LTSBikePlan/data/MonterealeValce_Cop.tif:
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 524, 'height': 412, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affin